In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [ ]:
df=pd.read_csv("/content/cardiovascular.csv",sep=";")

In [ ]:
df=df.rename({"obesity":"bmi"},axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 462 entries, 0 to 461
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ind        462 non-null    int64  
 1   sbp        462 non-null    int64  
 2   tobacco    462 non-null    float64
 3   ldl        462 non-null    float64
 4   adiposity  462 non-null    float64
 5   famhist    462 non-null    object 
 6   typea      462 non-null    int64  
 7   bmi        462 non-null    float64
 8   alcohol    462 non-null    float64
 9   age        462 non-null    int64  
 10  chd        462 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 39.8+ KB


In [ ]:
df=df.drop(columns=["ind","tobacco","adiposity","ldl","typea","alcohol",])

In [ ]:
fam_dict={i:k for k,i in enumerate(df["famhist"].unique())}
df["famhist"]=df["famhist"].apply(lambda x: fam_dict[x])

In [ ]:
mean_sbp=df["sbp"].mean()
std_sbp=df["sbp"].std()
mean_bmi=df["bmi"].mean()
std_bmi=df["bmi"].std()
mean_age=df["age"].mean()
std_age=df["age"].std()

In [ ]:
df["sbp"]=df["sbp"].apply(lambda x: (x-df["sbp"].mean())/df["sbp"].std())
df["bmi"]=df["bmi"].apply(lambda x: (x-df["bmi"].mean())/df["bmi"].std())
df["age"]=df["age"].apply(lambda x: (x-df["age"].mean())/df["age"].std())

In [ ]:
df

,sbp,famhist,bmi,age,chd
0,1.057417,0,-0.176594,0.628654,1
1,0.276789,1,0.670646,1.381617,1
2,-0.991731,0,0.734723,0.217947,0
3,1.545310,0,1.411091,1.039361,1
4,-0.211103,0,-0.012842,0.423301,1
...,...,...,...,...,...
457,3.692037,1,0.570971,1.039361,0
458,2.130781,1,0.608942,0.628654,1
459,-1.479624,1,-1.413043,0.834008,0
460,-0.991731,1,0.309916,-0.192760,0


In [ ]:
msk = np.random.rand(len(df)) < 0.8
trn=df[msk]
tst_val=df[~msk]
msk=np.random.rand(len(tst_val))<0.5
tst=tst_val[msk]
val=tst_val[~msk]

In [ ]:
x_trn=trn.drop(columns=["famhist","chd"]).values
fm_x_trn=to_categorical(trn["famhist"].values)
y_trn=trn["chd"].values

In [ ]:
x_trn

array([[ 1.05741729, -0.17659445,  0.62865426],
       [ 0.27678925,  0.67064592,  1.38161701],
       [ 1.54530982,  1.41109128,  1.03936121],
       ...,
       [-1.47962385, -1.41304329,  0.83400774],
       [-0.99173133,  0.30991612, -0.19275965],
       [-0.30868179, -2.69221012,  0.2179473 ]])

In [ ]:
x_tst=tst.drop(columns=["famhist","chd"]).values
fm_x_tst=to_categorical(tst["famhist"].values)
y_tst=tst["chd"].values

In [ ]:
x_val=val.drop(columns=["famhist","chd"]).values
fm_x_val=to_categorical(val["famhist"].values)
y_val=val["chd"].values

In [ ]:
input_=tf.keras.layers.Input(shape=(x_trn.shape[1],))
input_fm=tf.keras.layers.Input(shape=(2,))
concat_=tf.keras.layers.Concatenate()([input_,input_fm])
dens_=tf.keras.layers.Dense(16,activation="relu")(input_)
dens_2=tf.keras.layers.Dense(16,activation="relu")(dens_)

res=tf.keras.layers.Dense(1,activation="sigmoid")(dens_2)

In [ ]:
model=tf.keras.Model(inputs=[input_,input_fm],outputs=res)

In [ ]:
model.compile(optimizer="adam",loss=tf.keras.losses.BinaryCrossentropy(),metrics=["accuracy"])

In [ ]:
model.summary()

Model: "model_38"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_79 (InputLayer)           [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense_139 (Dense)               (None, 16)           64          input_79[0][0]                   
__________________________________________________________________________________________________
dense_140 (Dense)               (None, 16)           272         dense_139[0][0]                  
__________________________________________________________________________________________________
input_80 (InputLayer)           [(None, 2)]          0                                            
___________________________________________________________________________________________

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        # "no longer improving" being further defined as "for at least 10 epochs"
        restore_best_weights=True,
        patience=60,
        verbose=1,
    )
]

In [ ]:
model.fit(x=[x_trn,fm_x_trn],y=y_trn,batch_size=256,epochs=500,validation_data=([x_val,fm_x_val],y_val),callbacks=callbacks)

Epoch 1/500
2/2 [==============================] - 1s 149ms/step - loss: 0.6697 - accuracy: 0.6055 - val_loss: 0.6742 - val_accuracy: 0.5769
Epoch 2/500
2/2 [==============================] - 0s 22ms/step - loss: 0.6646 - accuracy: 0.6301 - val_loss: 0.6687 - val_accuracy: 0.6154
Epoch 3/500
2/2 [==============================] - 0s 20ms/step - loss: 0.6599 - accuracy: 0.6466 - val_loss: 0.6638 - val_accuracy: 0.6346
Epoch 4/500
2/2 [==============================] - 0s 20ms/step - loss: 0.6555 - accuracy: 0.6603 - val_loss: 0.6594 - val_accuracy: 0.6346
Epoch 5/500
2/2 [==============================] - 0s 22ms/step - loss: 0.6513 - accuracy: 0.6685 - val_loss: 0.6554 - val_accuracy: 0.6538
Epoch 6/500
2/2 [==============================] - 0s 20ms/step - loss: 0.6473 - accuracy: 0.6521 - val_loss: 0.6516 - val_accuracy: 0.6346
Epoch 7/500
2/2 [==============================] - 0s 24ms/step - loss: 0.6437 - accuracy: 0.6658 - val_loss: 0.6480 - val_accuracy: 0.6346
Epoch 8/500
2/2 [==

In [ ]:
model.evaluate(x=[x_tst,fm_x_tst],y=y_tst)

2/2 [==============================] - 0s 6ms/step - loss: 0.5903 - accuracy: 0.6667


[0.5902595520019531, 0.6666666865348816]

In [ ]:
model.evaluate(x=[x_val,fm_x_val],y=y_val)

2/2 [==============================] - 0s 5ms/step - loss: 0.5530 - accuracy: 0.6346


[0.5529981255531311, 0.6346153616905212]

In [ ]:
model.evaluate(x=[x_trn,fm_x_trn],y=y_trn)

12/12 [==============================] - 0s 1ms/step - loss: 0.5398 - accuracy: 0.7096


[0.5397560596466064, 0.7095890641212463]

In [ ]:
model.save("model.h5")

In [ ]:
spb=(float(input("systolic: "))-mean_sbp)/std_sbp #120
height=float(input("height: ")) #22.13
weight=float(input("weight: ")) 
age=(int(input("age: "))-mean_age)/std_age #22
fam=bool(input("Are there any relative in your family has cardiovascular?: (0 for not) (1 for yes) "))
bmi=((weight/(height*0.01)**2)-mean_bmi)/std_bmi
x=[np.asarray([[spb,bmi,age]]),to_categorical([fam])]
print ("your risk for having cardiovascular disease is :"+str(model.predict(x)))

systolic: 120
height: 180
weight: 200
age: 20
Are there any relative in your family has cardiovascular?: (0 for not) (1 for yes) 1
your risk for having cardiovascular disease is :[[0.6624554]]
